<a href="https://colab.research.google.com/github/cesar-claros/synergistic/blob/master/imdb_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install gpytorch

In [ ]:
! sudo apt-get install texlive-latex-recommended #1
! sudo apt-get install dvipng texlive-fonts-recommended #2
! wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip #3
! unzip type1cm.zip -d /tmp/type1cm #4
! cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins  #5
! sudo mkdir /usr/share/texmf/tex/latex/type1cm #6
! sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm #7
! sudo texhash #8

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!cp -r "drive/My Drive/NIPS2020/auxfunc/" .
!cp -r "drive/My Drive/NIPS2020/datasets/" .
!cp -r "drive/My Drive/NIPS2020/style/" .
!cp -r "drive/My Drive/NIPS2020/runs/" .

In [ ]:
%load_ext tensorboard

In [6]:
# Imports
import io #Used as buffer
import sys
import matplotlib
import tensorflow as tf # Keras model for MNIST 
# matplotlib.use('qt5Agg')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import auxfunc.sigfunc as sgn
import seaborn as sns
import torch
from scipy.stats import entropy, spearmanr
from sklearn import model_selection, svm, ensemble, linear_model, pipeline, \
      tree, neighbors, discriminant_analysis, gaussian_process, preprocessing, impute, decomposition
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern
plt.style.use(['ggplot','style/style.mplstyle'])

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
#%%
# Define LSTM architecture
def LSTM_model(top_words,emb_vector_length,max_review_length):
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(top_words,emb_vector_length,input_length=max_review_length))
    model.add(tf.keras.layers.LSTM(100))
    model.add(tf.keras.layers.Dense(
            units=1, kernel_initializer='normal', activation='sigmoid'
    ))
    return model

In [8]:
#%%
# Signaling function fitting and evaluation
def signalingFunction(X_train, y_train, y_train_pred_th, X_val, y_val, y_val_pred_th, X_test, y_test, y_test_pred_th, kernel='exponential', norm='l01'):
    # X_train, X_val should be scaled
    # Fit signaling function 
    exp = sgn.signaling(norm=norm) # idx = [train,test,val]
    exp.fit(X_train, y_train, y_train_pred_th, kernel=kernel, n_iter=500, lr=0.01)
    table_val = exp.evaluate(X_val, y_val, y_val_pred_th, rule_grid=np.linspace(0,3,30, endpoint=False))
    table_test = exp.test(X_test, y_test, y_test_pred_th, table_val['rule'].to_numpy(), table_val['eta'].to_numpy())
    table = pd.concat([table_val,table_test],axis=1)
    return table, exp

In [9]:
#%%
# Initialize model
def init_model(input_dim, objective):
    model = MLP_model(input_dim=input_dim, objective=objective)
    if objective=='svm':
        loss = tf.keras.losses.hinge
        metric = ['hinge']

    elif objective=='softmax':
        loss = tf.keras.losses.binary_crossentropy
        metric = ['accuracy']

    model.compile(loss=loss,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=metric)
    # model.summary()
    print('loss={}'.format(loss.__name__))
    return model

In [10]:
#%%
# Soft and thresholded output predictions
def pred_output(model, X, objective):
    if objective=='svm':
        y_pred_soft = model.predict(X)
        y_pred_th = np.array([1 if i[0] >= 0 else 0 for i in y_pred_soft])
    elif objective=='softmax':
        y_pred_soft = model.predict(X)
        y_pred_th = np.array([1 if i[0] > 0.5 else 0 for i in y_pred_soft])
    return y_pred_soft, y_pred_th

In [11]:
#%%
# Jaccard similarity index
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [12]:
#%%
# Baseline comparison
def baselineCriteria(y_val, y_val_pred_soft, y_val_pred_th, y_test, y_test_pred_soft, y_test_pred_th, table, exp, clf):
      if clf=='svm':
          direction = 'closer'
          crit_val = np.abs(y_val_pred_soft.ravel())
          crit_test = np.abs(y_test_pred_soft.ravel())
      else:
          direction = 'further'
          p_val = np.concatenate((y_val_pred_soft,1-y_val_pred_soft),axis=1)
          crit_val = entropy(p_val, axis=1, base=2)
          p_test = np.concatenate((y_test_pred_soft,1-y_test_pred_soft),axis=1)
          crit_test = entropy(p_test, axis=1, base=2)
      
      critFunc = sgn.critEvaluation(norm='l01',direction=direction)
      d_val = critFunc.evaluate(y_val, y_val_pred_th, crit_val)
      d_test = critFunc.test(y_test, y_test_pred_th, crit_test, d_val['thresh'].to_numpy())
      crit_table = pd.concat([d_val,d_test],axis=1)

      gamma = table['rule'].to_numpy().reshape(-1,1)
      f_test = exp.gpr_mean_test + gamma*np.sqrt(exp.gpr_var_test)
      eta = table['eta'].to_numpy().reshape(-1,1)
      theta = crit_table['thresh'].to_numpy().reshape(-1,1)
      if direction == 'closer':
        f_mask, f_idx = np.nonzero(f_test>eta)
      else:
        f_mask, f_idx = np.nonzero(f_test<eta)
      crit_mask, crit_idx = np.nonzero(crit_test.reshape(1,-1)<theta)
      print(list(np.unique(f_mask)))
      print(list(np.unique(crit_mask)))
      print(f_test.shape[0])
      shared = set(list(np.unique(f_mask))).intersection(set(list(np.unique(crit_mask))))
      J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) if i in shared else np.nan for i in range(f_test.shape[0])]
      # if (list(np.unique(f_mask))==list(np.unique(crit_mask))):
      #   J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) for i in np.unique(f_mask)]
      # else:
      #   shared = set(a).intersection(set(b))
      #   union = set(a).union(set(b))
      #   J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) if i in shared else np.nan  for i in union]
      crit_table['jaccard']=J
      Sp = [spearmanr(f_test[i,:],crit_test)[0] for i in range(f_test.shape[0])]
      crit_table['spearman'] = Sp
      crit_table['gamma'] = gamma
      return crit_table

In [13]:
# %%
# INITIALIZATION
# ==============
# EXPERIMENT SETUP
# ================
top_words = 5000
max_review_length = 500
emb_vector_length = 32
# Load data set
(Data_X, Data_y), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=top_words)

Data_X = tf.keras.preprocessing.sequence.pad_sequences(Data_X, maxlen=max_review_length)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_review_length)
print("Number of original training examples:", len(Data_X))
# For reproducibility
tf.random.set_seed(54321)
np.random.seed(12345)
torch.manual_seed(0)

17465344/17464789 [==============================] - 0s 0us/step
Number of original training examples: 25000


In [14]:
#%%
# Assign labels
report_table = []
report_criteria = []
report_plot = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=123)
clf = 'svm'
addPredictions = False
applyPCA = False
for sample, test in kf.split(Data_X):
    sample = sample[:12500]
    test = test[:2000]
    X = Data_X[sample]
    y = Data_y[sample]
    X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.20, random_state=123)
    X_test = Data_X[test]
    y_test = Data_y[test]

    # TRAINING MODEL
    model = init_model(input_dim=X.shape[1], objective=clf)
    model.fit(X_train, y_train, batch_size=64, epochs=5, verbose=0, validation_data=(X_val, y_val))
    X_test = scaleX.transform(imputeX.transform(X_test))

    y_train_pred_soft, y_train_pred_th = pred_output(model, X_train, clf)
    print('accuracy(Train)={}'.format(np.sum(y_train==y_train_pred_th)/np.size(y_train)))
    y_val_pred_soft, y_val_pred_th = pred_output(model, X_val, clf)
    y_test_pred_soft, y_test_pred_th = pred_output(model, X_test, clf)

    layer_outputs = [layer.output for layer in model.layers] 
    activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
    X_train_GP = activation_model.predict(X_train)[1]
    X_val_GP = activation_model.predict(X_val)[1]
    X_test_GP = activation_model.predict(X_test)[1]

    if addPredictions:
            # Add predictions
            X_train_GP = np.concatenate((X_train, y_train_pred_soft), axis=1)
            X_val_GP = np.concatenate((X_val, y_val_pred_soft), axis=1)
            X_test_GP = np.concatenate((X_test, y_test_pred_soft), axis=1)
    scaleX_GP = preprocessing.StandardScaler().fit(np.concatenate((X_train_GP, X_val_GP), axis=0))
    X_train_GP = scaleX_GP.transform(X_train_GP)
    X_val_GP = scaleX_GP.transform(X_val_GP)
    X_test_GP = scaleX_GP.transform(X_test_GP)
    if applyPCA:
            pca_GP = decomposition.PCA(.99).fit(np.concatenate((X_train_GP, X_val_GP), axis=0)) # set percentage of energy preserved by PCA
            # Apply PCA transform to all sets
            X_train_GP = pca_GP.transform(X_train_GP)
            X_val_GP = pca_GP.transform(X_val_GP)
            X_test_GP = pca_GP.transform(X_test_GP)

    table, exp = signalingFunction(X_train_GP, y_train, y_train_pred_th, X_val_GP, y_val, y_val_pred_th, X_test_GP, y_test, y_test_pred_th)
    report_table.append(table)
    # Baseline for comparison
    crit_table = baselineCriteria(y_val, y_val_pred_soft, y_val_pred_th, y_test, y_test_pred_soft, y_test_pred_th, table, exp, clf)
    report_criteria.append(crit_table)
    del(model)

NameError: ignored

In [ ]:
#reverse lookup
INDEX_FROM = 3
word_to_id = tf.keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}

In [ ]:
# Plot signaled instances
rho = 0.05
rule = table.loc[table.rho_user == rho]['rule'].to_numpy()
eta = table.loc[table.rho_user == rho]['eta'].to_numpy()
f_test = exp.gpr_mean_test+rule*np.sqrt(exp.gpr_var_test)
top_n = 8 # Top n selected instances in test set
top_f_idx = np.argpartition(f_test, -top_n)[-top_n:]

In [ ]:
for i in top_f_idx:
  print(' '.join(id_to_word[id] for id in X_test[i,:] )+\
        '\n y={}'.format(y_test[i])+', y_pred={}'.format(y_test_pred_th[i])+\
        ', H={}'.format(crit_test[i])+'\n')

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> in <UNK> a film tales of manhattan told a set of stories that were basically <UNK> but tied together with a suit of <UNK> evening wear each story began when the <UNK> were passed from one owner charles <UNK> for instance to another <UNK> <UNK> <UNK> <UNK> a superior film and a great western has a similar plot twist initially it is about how jimmy stewart is seeking stephen <UNK> <UNK> for some deadly <UNK> but in the course of the film the two men get into a shooting contest the prize given by <UNK> <UNK> <UNK> will <UNK> being one of the new <UNK> <UNK> stewart barely beats out <UNK> but the gun is stolen from stewart and the chase is on br br the gun passes from hand to hand including john <UNK> as an arrogant <UNK> who <UNK> does not know when to stop being arrogant to rock hudson in a surprising role and a brief one at that to charles <UNK> to dan <UNK> as the <UNK> deadly and psychotic <UNK> johnny dea

In [ ]:
top_H_idx = np.argpartition(crit_test, -top_n)[-top_n:]
for i in top_H_idx:
  print(' '.join(id_to_word[id] for id in X_test[i,:] )+\
        '\n y={}'.format(y_test[i])+', y_pred={}'.format(y_test_pred_th[i])+\
        ', H={}'.format(crit_test[i])+'\n')

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

In [ ]:
report_table_concat = pd.concat(report_table)
table_by_row_index = report_table_concat.groupby(report_table_concat.index)
report_table_mean = table_by_row_index.mean()
report_table_std = table_by_row_index.std()

report_criteria_concat = pd.concat(report_criteria)
table_by_row_index = report_criteria_concat.groupby(report_criteria_concat.index)
report_criteria_mean = table_by_row_index.mean()
report_criteria_std = table_by_row_index.std()

In [ ]:
report_table_mean

,rule,rho_user,error_val,L_val,%reduction_val,eta,p_value,check,budget,error_test,L_test,%reduction_test
0,1.06,0.01,0.00560,0.14192,3.786,0.820517,1.596467e-04,0.01,0.010,0.0049,0.1471,3.218
1,1.20,0.05,0.02472,0.12280,16.776,0.672585,1.438608e-20,0.05,0.054,0.0247,0.1273,16.284
2,0.84,0.10,0.04568,0.10184,31.026,0.448278,2.111505e-39,0.10,0.102,0.0457,0.1063,30.058
3,1.26,0.15,0.06064,0.08688,41.244,0.423398,1.133743e-41,0.15,0.154,0.0634,0.0886,41.738
4,0.12,0.20,0.07376,0.07376,50.104,0.136592,5.266706e-43,0.20,0.200,0.0775,0.0745,50.980


In [ ]:
report_criteria_mean

,rho_user,error_val,L_val,%reduction_val,thresh,budget,error_test,L_test,%reduction_test
0,0.01,0.00488,0.14264,3.306,0.998604,0.012,0.0057,0.1463,3.672
1,0.05,0.02320,0.12432,15.728,0.964553,0.054,0.0242,0.1278,15.878
2,0.10,0.04408,0.10344,29.916,0.885287,0.102,0.0452,0.1068,29.700
3,0.15,0.06088,0.08664,41.324,0.774637,0.152,0.0654,0.0866,43.020
4,0.20,0.07608,0.07144,51.648,0.668580,0.206,0.0825,0.0695,54.300


In [ ]:
report_table_std

,rule,rho_user,error_val,L_val,%reduction_val,eta,p_value,check,budget,error_test,L_test,%reduction_test
0,1.176010,0.0,0.001265,0.007034,0.797076,0.219716,3.569270e-04,0.0,0.000000,0.001817,0.009283,1.190869
1,1.104536,0.0,0.003025,0.007595,2.114800,0.264298,3.187349e-20,0.0,0.005477,0.002564,0.009425,1.828026
2,1.221884,0.0,0.002918,0.007857,2.533146,0.322871,4.721467e-39,0.0,0.008367,0.006535,0.008526,3.697022
3,1.047855,0.0,0.004495,0.010188,4.406856,0.203426,2.432024e-41,0.0,0.008944,0.006628,0.008569,3.804684
4,0.178885,0.0,0.005127,0.009045,4.302619,0.043197,1.177661e-42,0.0,0.010000,0.010186,0.009670,5.632673


In [ ]:
report_criteria_std

,rho_user,error_val,L_val,%reduction_val,thresh,budget,error_test,L_test,%reduction_test
0,0.0,0.000996,0.007357,0.659454,0.000779,0.004472,0.003054,0.007032,1.738094
1,0.0,0.002871,0.006832,1.788413,0.016886,0.005477,0.003347,0.007023,1.419972
2,0.0,0.002834,0.007164,2.081797,0.053454,0.008367,0.005251,0.006496,2.410187
3,0.0,0.004625,0.007785,3.379797,0.081735,0.008367,0.005067,0.005878,1.744506
4,0.0,0.005370,0.008341,4.106479,0.093708,0.008944,0.006225,0.006567,2.760860
